# Resolve did:btc1 from local Bitcoin node in regtest

In [1]:
import sys
import os

notebooks_path = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the Notebooks directory to the sys.path
sys.path.append(notebooks_path)

In [2]:
sidecar_data = {'did': 'did:btc1:regtest:k1q2aehd6fnzu2p334mm5r38sfw70cpyklwg88yr9ct3u25wfdj3rkgf8sc6r',
 'signals': {'082a88274939c2701a33dc6dc27b8af063bc26f749cad83cb760384c0bf1a4d2': {'updatePayload': {'@context': ['https://w3id.org/security/v2',
     'https://w3id.org/zcap/v1',
     'https://w3id.org/json-ld-patch/v1'],
    'patch': [{'op': 'add',
      'path': '/service/3',
      'value': {'id': '#linked-domain',
       'type': 'LinkedDomains',
       'serviceEndpoint': 'https://contact-me.com'}}],
    'sourceHash': '3PSk7NgH5jutsWjinEDJv1FpvgMoYNZCbQdnb8HqBLMX',
    'targetHash': '9HjW9vPH4FApyEEqhjCV5L7jdJeKhbP9VJDWW7xFRmtr',
    'targetVersionId': 2,
    'proof': {'type': 'DataIntegrityProof',
     'cryptosuite': 'schnorr-secp256k1-jcs-2025',
     'verificationMethod': 'did:btc1:regtest:k1q2aehd6fnzu2p334mm5r38sfw70cpyklwg88yr9ct3u25wfdj3rkgf8sc6r#initialKey',
     'proofPurpose': 'capabilityInvocation',
     '@context': ['https://w3id.org/security/v2',
      'https://w3id.org/zcap/v1',
      'https://w3id.org/json-ld-patch/v1'],
     'proofValue': 'z4S5nEKh4Aq7abZZVBnYbHy5mTWbBXDavCXmKHGH8EjhkGtv8BcuCfbHYwLxMEja9DzSXWCntn3V3nNZTzaMmn1US'}}}}}

In [3]:
did_to_resolve = "did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65"

resolutionOptions = {"sidecarData": sidecar_data}

## Parse did:btc1 identifier

In [4]:
import re
match = re.search(r"^did:btc1:(?:(\d+):)?(?:(mainnet|signet|testnet|regtest):)?((k1|x1)[023456789acdefghjklmnpqrstuvwxyz]*)$", did_to_resolve)
groups = match.groups()
groups

(None,
 'regtest',
 'k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65',
 'k1')

In [5]:
identifierComponents = {}

identifierComponents["version"] = groups[0] or 1
identifierComponents["network"] = groups[1] or "mainnet"




In [6]:
from libbtc1.bech32 import decode_bech32_identifier

bech32_encoding = groups[2]
hrp, genesisBytes = decode_bech32_identifier(bech32_encoding)

In [7]:
identifierComponents["hrp"] = hrp
identifierComponents["genesisBytes"] = genesisBytes

In [8]:
identifierComponents

{'version': 1,
 'network': 'regtest',
 'hrp': 'k',
 'genesisBytes': b"\x02\x9a\xd5\xf6\xa8]'\xeei\xb13\xae\xd2s\xb4\xf2\xf5\xd7\x0e\xd4\xa7\x16u\x01\x9c\x1av\xf0Lf5&\xef"}

## Resolve Initial Document

This algorithm specifies how to resolve an initial DID document and validate
it against the `identifier` for a specific **did:btc1**. The algorithm takes as
inputs a **did:btc1** `identifier`, `identifierComponents` object and a
`resolutionsOptions` object. This algorithm returns a valid `initialDocument`
for that identifier.

In [9]:
print(identifierComponents["hrp"])
if identifierComponents["hrp"] == "k":
    print("Deterministically Generate Initial DID Document")

k
Deterministically Generate Initial DID Document


In [10]:
from libbtc1.did import resolve_deterministic

initial_document = resolve_deterministic(did_to_resolve, identifierComponents)

In [12]:
initial_document

{'id': 'did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65',
 '@context': ['https://www.w3.org/ns/did/v1', 'https://did-btc1/TBD/context'],
 'verificationMethod': [{'id': '#initialKey',
   'type': 'Multikey',
   'controller': 'did:btc1:regtest:k1q2ddta4gt5n7u6d3xwhdyua57t6awrk55ut82qvurfm0qnrxx5nw7vnsy65',
   'publicKeyMultibase': 'z66PwJnYvwJLhGrVc8vcuUkKs99sKCzYRM2HQ2gDCGTAStHk'}],
 'authentication': ['#initialKey'],
 'assertionMethod': ['#initialKey'],
 'capabilityInvocation': ['#initialKey'],
 'capabilityDelegation': ['#initialKey'],
 'service': [{'id': '#initial_p2pkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:ms4wr9JokfjLPtBVHpMefXS6domSgEdHXf'},
  {'id': '#initial_p2wpkh',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1q06m9yn2kxgxg2mara55667958d0a6s2ksdc3dd'},
  {'id': '#initial_p2tr',
   'type': 'SingletonBeacon',
   'serviceEndpoint': 'bitcoin:bcrt1psajngvjme0fkzv83m2ys80dhmya3nghph4h478ad2z5q564049jsuu7sng'}]

## Resolve Target Document

This algorithm resolves a DID document from an initial document by walking the
Bitcoin blockchain to identify Beacon Signals that announce DID Update Payloads
applicable to the **did:btc1** identifier being resolved. The algorithm takes
in an `initialDocument` and a set of `resolutionOptions`. The algorithm returns
a valid `targetDocument` or throws an error.

1. If `resolutionOptions.versionId` is not null, set `targetVersionId` to
   `resolutionOptions.versionId`.
1. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to
   `resolutionOptions.versionTime`.
1. Set `targetBlockheight` to the result of passing `targetTime` to the algorithm
   [Determine Target Blockheight].
1. Set `sidecarData` to `resolutionOptions.sidecarData`.
1. Set `currentVersionId` to 1.
1. If `currentVersionId` equals `targetVersionId` return `initialDocument`.
1. Set `updateHashHistory` to an empty array.
1. Set `contemporaryBlockheight` to 0.
1. Set `contemporaryDIDDocument` to the `initialDocument`.
1. Set `targetDocument` to the result of calling the [Traverse Blockchain History]
   algorithm passing in `contemporaryDIDDocument`, `contemporaryBlockheight`,
   `currentVersionId`, `targetVersionId`, `targetBlockheight`, `updateHashHistory`, 
   and `sidecarData`.
1. Return `targetDocument`.

### 1. If resolutionOptions.versionId is not null, set targetVersionId to resolutionOptions.versionId.
### 2. Else if `resolutionOptions.versionTime` is not null, set `targetTime` to `resolutionOptions.versionTime`.

In [18]:
versionId = resolutionOptions.get("versionId")
versionTime = resolutionOptions.get("versionTime")
targetTime = None
targetVersionId = None
if versionId:
    targetVersionId = versionId
elif versionTime:
    targetTime = versionTime

### 3. Set targetBlockheight to the result of passing targetTime to the algorithm [Determine Target Blockheight].

## Determine Target Blockheight

This algorithm takes in an OPTIONAL Unix `targetTime` and returns a Bitcoin
`blockheight`.

In [27]:
required_confirmations = 6
if targetTime:
    raise "NotImplemented"
else:
    print("find latest block with at least x confimations")



find latest block with at least x confimations


In [28]:
from bitcoinrpc import BitcoinRPC
rpc = BitcoinRPC.from_config("http://localhost:18443", ("polaruser", "polarpass"))
await rpc.getconnectioncount()

0

In [34]:
best_blockhash = await rpc.acall("getbestblockhash", {})
bestblock = await rpc.acall("getblock", {"blockhash": best_blockhash})

In [35]:
confirmations = bestblock["confirmations"]
bestblock_height = bestblock["height"]

In [36]:
targetblockheight = bestblock_height + (confirmations - required_confirmations)

In [37]:
targetblockheight

128

### 4. Set `sidecarData` to `resolutionOptions.sidecarData`.

In [39]:
sidecarData = resolutionOptions["sidecarData"]

### 5. Set `currentVersionId` to 1.


In [40]:
currentVersionId = 1

### 6. If `currentVersionId` equals `targetVersionId` return `initialDocument`.

In [41]:
if currentVersionId == targetVersionId:
    print("returning", initial_document)

### 7. Set `updateHashHistory` to an empty array.

In [42]:
updateHashHistory = []

### 8. Set `contemporaryBlockheight` to 0.